In [4]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split, cross_val_score

In [5]:
full_data = pd.read_csv("joined_data.csv")
full_data
data = full_data[full_data.columns[4:]] # Dropping first columns. 
data

,core_goals,core_assists,core_saves,core_shots,core_score,demo_inflicted,demo_taken,advanced_rating
0,3,0,4,9,1119,2,2,0.863590
1,2,5,2,13,1209,2,0,1.350500
2,4,2,2,12,1077,2,5,1.232970
3,1,1,8,5,1086,1,2,0.806692
4,3,0,4,4,948,3,3,1.077983
...,...,...,...,...,...,...,...,...
25479,6,8,10,18,2886,6,5,1.306242
25480,3,4,6,12,1719,7,9,0.755061
25481,6,3,11,11,2462,8,5,1.148616
25482,6,2,5,27,2251,10,10,0.949304


In [7]:
mp = pd.read_csv("matches_by_players.csv")
mp
#player_list = {mp.iloc[row]["player_id"]:mp.iloc[row]["player_tag"] for row in range(len(mp))}
player_list = dict(zip(mp.player_id, mp.player_tag))

In [8]:
pd.DataFrame(list(zip(mp.player_id, mp.player_tag, mp.match_id)))

,0,1,2
0,5f3d8fdd95f40596eae2412e,Amphis,6159ad3d143c37878b2384a9
1,5f3d8fdd95f40596eae23e01,Torsos,6159ad3d143c37878b2384a9
2,5f3d8fdd95f40596eae23e53,Express,6159ad3d143c37878b2384a9
3,5f7ca648ea8a0f0714fb9a20,Laxin,6159ad3d143c37878b2384a9
4,5f3d8fdd95f40596eae24503,Baked Potato,6159ad3d143c37878b2384a9
...,...,...,...
25618,5f3d8fdd95f40596eae23f8f,Maxeew,62a3988cda9d7ca1c7bb22ba
25619,5f3d8fdd95f40596eae23f3f,Abscrazy,62a3988cda9d7ca1c7bb22ba
25620,5f9c7cde5246bf27936b4572,mikan,62a3988cda9d7ca1c7bb22ba
25621,5f3d8fdd95f40596eae2414a,Burn,62a3988cda9d7ca1c7bb22ba


In [9]:
# Splitting Data
Y = data["advanced_rating"]
X = data[['core_goals','core_assists', 'core_saves', 'core_shots', 'core_score','demo_inflicted', 'demo_taken']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=404)


In [10]:
model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(f'model intercept is {model.intercept_} and model coefficients are {model.coef_}')
print(f'model score = {model.score(X_test, Y_test)}')
print(f'mse = {metrics.mean_squared_error(Y_test, y_pred)}')


model intercept is 0.6559702384960422 and model coefficients are [ 0.18193362  0.10851521  0.04843667  0.00215491 -0.00045149 -0.00763981
 -0.01182257]
model score = 0.7911050337979552
mse = 0.02452801704841751


In [11]:
modelR = linear_model.Ridge()
modelR.fit(X_train,Y_train)
modelR.score(X_test, Y_test) # 
y_pred = model.predict(X_test)
print(f'model intercept is {model.intercept_} and model coefficients are {model.coef_}')
print(f'model score = {model.score(X_test, Y_test)}')
print(f'mse = {metrics.mean_squared_error(Y_test, y_pred)}')


model intercept is 0.6559702384960422 and model coefficients are [ 0.18193362  0.10851521  0.04843667  0.00215491 -0.00045149 -0.00763981
 -0.01182257]
model score = 0.7911050337979552
mse = 0.02452801704841751


make dataframe for coefficients and features

In [12]:
print(f'model intercept is {modelR.intercept_} and model coefficients are {modelR.coef_}')
print(modelR.feature_names_in_)

model intercept is 0.6559595152814083 and model coefficients are [ 0.18192036  0.10850757  0.04842838  0.00215379 -0.0004514  -0.00764043
 -0.0118235 ]
['core_goals' 'core_assists' 'core_saves' 'core_shots' 'core_score'
 'demo_inflicted' 'demo_taken']


In [13]:
modelL = linear_model.Lasso(alpha = 0.01)
modelL.fit(X_train, Y_train)
modelL.score(X_test, Y_test)

0.7879866128603924

In [14]:
print(f'model intercept is {modelL.intercept_} and model coefficients are {modelL.coef_}')

model intercept is 0.6371271178634823 and model coefficients are [ 0.16385644  0.09627452  0.03511792 -0.         -0.00031597 -0.00733571
 -0.01163212]


In [15]:
pd.DataFrame({"Ridge Model":modelR.coef_, "Standard Linear":model.coef_, "Lasso":modelL.coef_}, index = X_train.columns) 

,Ridge Model,Standard Linear,Lasso
core_goals,0.181920,0.181934,0.163856
core_assists,0.108508,0.108515,0.096275
core_saves,0.048428,0.048437,0.035118
core_shots,0.002154,0.002155,-0.000000
core_score,-0.000451,-0.000451,-0.000316
demo_inflicted,-0.007640,-0.007640,-0.007336
demo_taken,-0.011824,-0.011823,-0.011632
